## Testing the linear program against DOSCOE

To test the logic of my linear program, I use cost inputs and other parameters from the Google DOSCOE paper and compare the results. Some elements of the code (ex. storage) are altered in order to compare the programs directly.

In [304]:
%reload_ext autoreload
%autoreload 2

import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting library
import datetime as dt
import pandas as pd

from ortools.linear_solver import pywraplp

In [305]:
import utils

In [371]:
#Changeable parameters.

battery_duration = 4
initial_state_of_charge = 0
timespan = 30
discount_rate = 0.06
gas_fuel_cost = 14
cost = "1"

In [372]:
doscoe_solver = pywraplp.Solver('HarborOptimization',
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

#Introduce objective object so we can refer to it in the for loop.
objective = doscoe_solver.Objective()

In [373]:
# Load generation profiles for nondispatchable resources (KWh generated each hour by 1 KW of capacity).
profiles = pd.read_csv('data/doscoe_profiles.csv')

In [374]:
profiles.head()

,DEMAND,SOLAR,WIND,COAL,COAL_CRYO,COAL_AMINE,NUCLEAR
0,29309.5,531.90,9697.45,57747,57747,57747,115494
1,30445.5,1.44,10613.87,57747,57747,57747,115494
2,32716.5,0.00,11480.82,57747,57747,57747,115494
3,34456.5,0.00,11863.41,57747,57747,57747,115494
4,34098.0,0.00,12546.18,57747,57747,57747,115494


In [375]:
resources = pd.read_csv('data/doscoe_resources.csv')
resources = resources[resources['resource'].str.contains('_0|_2') == False]
resources = resources.set_index('resource')     
resources.index = [resource.replace('_'+cost,'') for resource in resources.index]
resources


,legacy,existing_mw,dispatchable,capex,variable,heat_rate,CO2
COAL,n,NaN,n,3388938.68,23.30,0.00,0.86
COAL_AMINE,n,NaN,n,5693261.60,61.95,0.00,0.11
COAL_CRYO,n,NaN,n,4559261.60,42.87,0.00,0.10
HYDROPOWER,y,9590.0,y,217839.46,2.66,0.00,0.00
NGCC,n,NaN,y,1239031.77,1.99,6.20,0.33
NGCC_AMINE,n,NaN,y,2482557.85,7.08,6.20,0.04
NGCC_CRYO,n,NaN,y,1973557.85,7.08,6.20,0.04
NGCT,n,NaN,y,770633.27,10.64,8.55,0.45
NGCT_AMINE,n,NaN,y,2014159.35,15.73,8.55,0.06
NGCT_CRYO,n,NaN,y,1505159.35,15.73,8.55,0.05


In [376]:
# storage = pd.read_csv('data/storage_costs.csv')
# storage = storage.set_index('resource')
# storage

In [377]:
# outofbasin_emissions = pd.read_csv('data/outofbasin_emissions.csv')
# outofbasin_emissions.insert(0, 'datetime', harborgen.index)
# outofbasin_emissions = outofbasin_emissions.set_index('datetime')
# outofbasin_emissions

In [378]:
#Hourly monetized grid emissions for the whole LADWP grid will be used to quantify the emissions impacts of storage charging.
# whole_grid_emissions = pd.read_csv('data/whole_grid_emissions.csv')
# whole_grid_emissions.insert(0, 'datetime', harborgen.index)
# whole_grid_emissions = whole_grid_emissions.set_index('datetime')
# whole_grid_emissions

In [379]:
# Declare nameplate capacity variables for each resource in resource cost and storage cost dataframes.
capacity_vars = {}
for resource in resources.index:
    if resources.loc[str(resource)]['legacy'] == 'n':
        capacity = doscoe_solver.NumVar(0, doscoe_solver.infinity(), str(resource))
        capacity_vars[resource] = capacity
    else:
        max_hydro = resources.loc[str(resource)]['existing_mw']
        capacity = doscoe_solver.NumVar(0, max_hydro, str(resource))
        capacity_vars[resource] = capacity

# for resource in storage.index:
#     capacity = doscoe_solver.NumVar(0, doscoe_solver.infinity(), str(resource))
#     capacity_vars[resource] = capacity
    
capacity_vars

{'COAL': COAL,
 'COAL_AMINE': COAL_AMINE,
 'COAL_CRYO': COAL_CRYO,
 'HYDROPOWER': HYDROPOWER,
 'NGCC': NGCC,
 'NGCC_AMINE': NGCC_AMINE,
 'NGCC_CRYO': NGCC_CRYO,
 'NGCT': NGCT,
 'NGCT_AMINE': NGCT_AMINE,
 'NGCT_CRYO': NGCT_CRYO,
 'NUCLEAR': NUCLEAR,
 'SOLAR': SOLAR,
 'WIND': WIND}

In [380]:
#Create filtered dataframes for dispatchable and nondispatchable resources.
disp = resources.loc[resources['dispatchable'] == 'y']
nondisp = resources.loc[resources['dispatchable'] == 'n']
nondisp


,legacy,existing_mw,dispatchable,capex,variable,heat_rate,CO2
COAL,n,NaN,n,3388938.68,23.30,0.0,0.86
COAL_AMINE,n,NaN,n,5693261.60,61.95,0.0,0.11
COAL_CRYO,n,NaN,n,4559261.60,42.87,0.0,0.10
NUCLEAR,n,NaN,n,4667257.68,12.00,0.0,0.00
SOLAR,n,NaN,n,1356035.02,0.00,0.0,0.00
WIND,n,NaN,n,2181532.58,0.00,0.0,0.00


In [381]:
#Create a dictionary to hold a list for each dispatchable resource that keeps track of its hourly generation variables.
disp_gen = {}
for resource in disp.index:
    disp_gen[resource] = []

In [382]:
disp_gen

{'HYDROPOWER': [],
 'NGCC': [],
 'NGCC_AMINE': [],
 'NGCC_CRYO': [],
 'NGCT': [],
 'NGCT_AMINE': [],
 'NGCT_CRYO': []}

In [383]:
#Create a dictionary to hold a list for each storage resource that keeps track of its hourly charge variables.
# charge_vars = {}
# for resource in storage.index:
#     charge_vars[resource] = []

In [384]:
#Create a dictionary to hold a list for each storage resource that keeps track of its hourly discharge variables.
# discharge_vars = {}
# for resource in storage.index:
#     discharge_vars[resource] = []

In [385]:
#Creates a dictionary to track the hourly state of charge of each storage resource. Each value represents the state of charge at the start of each timestep.
# state_of_charge_vars = {}
# for resource in storage.index:
#     state_of_charge_vars[resource] = [] 

In [386]:
# growth_rate = 1 + discount_rate
# variable_cost = 10
# discounted_cost = variable_cost / pow(growth_rate, -timespan)            
# print(discounted_cost)


In [387]:
cost = 1
growth_rate = 1.0 + discount_rate
value_decay_1 = pow(growth_rate, -timespan)
value_decay_2 = pow(growth_rate, -1)
discounting_factor = cost * (1.0 - value_decay_1) / (1.0-value_decay_2)
print(discounting_factor)

14.59072102057877


In [388]:
hydro_limit = doscoe_solver.Constraint(0, 13808000)

#Loop through every hour, creating 1) hourly generation variables for each dispatchable resource, 2) hourly constraints, and 3) adding variable cost coefficients to each hourly generation variable.
for ind in profiles.index:
    
    #Summed generation from all resources must be equal or greater to demand in all hours.
    fulfill_demand = doscoe_solver.Constraint(profiles.loc[ind,'DEMAND'], doscoe_solver.infinity())
    
    #Create hourly charge and discharge variables for each storage resource and store in respective dictionaries. 
#     for resource in storage.index:
        
#         #Create hourly charge and discharge variables for each storage resource.
#         charge = doscoe_solver.NumVar(0, doscoe_solver.infinity(), '_charge'+ str(ind))
#         discharge = doscoe_solver.NumVar(0, doscoe_solver.infinity(), '_discharge'+ str(ind))
        
#         #Add variable cost of charging and monetized emissions to objective function.
#         variable_cost = whole_grid_emissions.loc[ind,'TOTAL/MWH']+ storage.loc[resource,'variable($/MWh)']
#         objective.SetCoefficient(charge, variable_cost)
        
#         #Limit hourly charge and discharge variables to storage max power (MW).
#         max_charge = doscoe_solver.Constraint(0, doscoe_solver.infinity())
#         max_charge.SetCoefficient(capacity_vars[resource], 1)
#         max_charge.SetCoefficient(charge, -1)
        
#         max_discharge = doscoe_solver.Constraint(0, doscoe_solver.infinity())
#         max_discharge.SetCoefficient(capacity_vars[resource], 1)
#         max_discharge.SetCoefficient(discharge, -1)
        
#         #Keep track of hourly charge and discharge variables by appending to lists for each storage resource.
#         charge_vars[resource].append(charge)
#         discharge_vars[resource].append(discharge)
        
#         #Hourly discharge variables of storage resources are incorporated into the fulfill demand constraint. If storage can only charge from portfolio resources, include the charge variable in this constraint.
#         efficiency = storage.loc[resource, 'efficiency']
#         fulfill_demand.SetCoefficient(discharge, efficiency)
#         #Include the line below if storage can only charge from portfolio resources.
#         #fulfill_demand.SetCoefficient(charge, -1)
        
#         #Creates hourly state of charge variable, representing the state of charge at the end of each timestep. 
#         state_of_charge = doscoe_solver.NumVar(0, doscoe_solver.infinity(), 'state_of_charge'+ str(ind))
        
#         #Temporal coupling of storage state of charge.
#         if harborgen.index.get_loc(ind) > 0:
#             temporal = doscoe_solver.Constraint(0, 0)
#             temporal.SetCoefficient(state_of_charge, -1)
#             temporal.SetCoefficient(discharge, -1)
#             temporal.SetCoefficient(charge, efficiency)
#             #Get the state of charge from previous timestep to include in the temporal coupling constraint.
#             previous_state = state_of_charge_vars[resource][-1]
#             temporal.SetCoefficient(previous_state, 1)
#         else: 
#             temporal = doscoe_solver.Constraint(initial_state_of_charge, initial_state_of_charge)
#             temporal.SetCoefficient(state_of_charge, 1)
#             temporal.SetCoefficient(discharge, 1)
#             temporal.SetCoefficient(charge, -efficiency)
        
#         #Add hourly state of charge variable to corresponding list for each storage resource.
#         state_of_charge_vars[resource].append(state_of_charge)
        
#         #Creates constraint setting max for storage state of charge.
#         max_storage = doscoe_solver.Constraint(0, doscoe_solver.infinity())
#         max_storage.SetCoefficient(state_of_charge, -1)
#         max_storage.SetCoefficient(capacity_vars[resource], battery_duration)
        
#         #Creates constraint ensuring that no net energy is supplied by storage (ending state of charge is equal to initial state of charge).
#         if harborgen.index.get_loc(ind) == len(harborgen)-1:
#             ending_state = doscoe_solver.Constraint(initial_state_of_charge, initial_state_of_charge)
#             ending_state.SetCoefficient(state_of_charge, 1)
            

    #Create generation variable for each dispatchable resource for every hour. Append hourly gen variable to the list for that resource, located in the disp_gen dictionary.
    #Create constraint that generation must be less than or equal to capacity for each dispatchable resource for all hours.
    for resource in disp.index:
        
        gen = doscoe_solver.NumVar(0, doscoe_solver.infinity(), '_gen'+ str(ind))
        disp_gen[resource].append(gen)
#         if resource == 'outofbasin':
#             # TODO: Incorporate transmission cost into variable cost for outofbasin option.
#             variable_cost = outofbasin_emissions.loc[ind,'TOTAL/MWH']+ disp.loc[resource,'variable']
#             objective.SetCoefficient(gen, variable_cost)
        if 'NG' in resource:
            variable_cost = disp.loc[resource,'variable']+ (disp.loc[resource,'heat_rate']* gas_fuel_cost)
        else:
            variable_cost = disp.loc[resource,'variable']

        objective.SetCoefficient(gen, variable_cost * discounting_factor)
            
        
        #Set coefficients for the hourly gen variables for the fulfill_demand constraint.
        fulfill_demand.SetCoefficient(gen, 1)
        
        #Set coefficients for dispatchable capacity variables and hourly gen variables for the max_gen = capacity constraint. 
        #For legacy resources, contrains maximum hourly generation to existing capacity.
        max_gen = doscoe_solver.Constraint(0, doscoe_solver.infinity())
        capacity = capacity_vars[resource]
        max_gen.SetCoefficient(capacity, 1)
        max_gen.SetCoefficient(gen, -1)
            
        if 'HYDRO' in resource:
            hydro_limit.SetCoefficient(gen, 1)
    
    #For each nondispatchable resource, set the coefficient of the capacity variable to its generation profile scaling factor. **Make sure units are aligned here (kw vs. mw capacities)
    for resource in nondisp.index: 
        capacity = capacity_vars[resource]
        profile_max = max(profiles[resource])
        coefficient = profiles.loc[ind, resource] / profile_max
        fulfill_demand.SetCoefficient(capacity, coefficient)
        
        variable_cost = nondisp.loc[resource,'variable'] * discounting_factor
        objective.SetCoefficient(capacity, variable_cost * coefficient)


In [389]:
# Amortized fixed costs (discounted over 30 years) are already incorporated into DOSCOE capex values.
for resource in resources.index:
    if resources.loc[str(resource)]['legacy'] == 'n':
        capex = resources.loc[resource, 'capex']
        objective.SetCoefficient(capacity_vars[resource], capex)
    else:
        capex = 14930 * discounting_factor
        objective.SetCoefficient(capacity_vars[resource], capex)
        
        
# for resource in storage.index:
#     capex = storage.loc[resource, 'capex ($/MW)']
#     objective.SetCoefficient(capacity_vars[resource], capex)

objective.SetMinimization()
status = doscoe_solver.Solve()
if status == doscoe_solver.OPTIMAL:
    print("Solver found optimal solution.")
    print("total cost =", objective.Value())

    total_capacity = 0
    for resource in capacity_vars:
        total_capacity = total_capacity + capacity_vars[resource].solution_value()
    for resource in capacity_vars:
        fraction_capacity = capacity_vars[resource].solution_value() / total_capacity
        print(str(capacity_vars[resource]) + ' fraction capacity =' + str(fraction_capacity))

    # Sum generation for each resource and print.
    total_gen = 0
    for resource in disp.index:
        summed_gen = 0
        for i_gen in disp_gen[str(resource)]:
            summed_gen += i_gen.solution_value()
        total_gen = total_gen + summed_gen

        
    ## Sum annual generation for nondispatchable resources.
    for resource in nondisp.index:
        profile_max = max(profiles[resource])
        summed_gen = sum(profiles[resource]) / profile_max
        capacity = capacity_vars[resource].solution_value()
        gen = summed_gen * capacity
        total_gen = total_gen + gen

    for resource in disp.index:
        summed_gen = 0
        for i_gen in disp_gen[str(resource)]:
            summed_gen += i_gen.solution_value()
        print(str(resource) + ' fraction generation =' + str(summed_gen / total_gen))  
        
    for resource in nondisp.index:
        profile_max = max(profiles[resource])
        summed_gen = sum(profiles[resource]) / profile_max
        capacity = capacity_vars[resource].solution_value()
        gen = summed_gen * capacity
        print(str(resource) + ' fraction generation =' + str(gen / total_gen)) 

    
else:
    print("Solver exited with error code {}".format(status))

Solver found optimal solution.
total cost = 132790450168.53685
COAL fraction capacity =0.576536632181332
COAL_AMINE fraction capacity =0.0
COAL_CRYO fraction capacity =0.0
HYDROPOWER fraction capacity =0.16606923303375068
NGCC fraction capacity =0.0
NGCC_AMINE fraction capacity =0.0
NGCC_CRYO fraction capacity =0.0
NGCT fraction capacity =0.25739413478491735
NGCT_AMINE fraction capacity =0.0
NGCT_CRYO fraction capacity =0.0
NUCLEAR fraction capacity =0.0
SOLAR fraction capacity =0.0
WIND fraction capacity =0.0
HYDROPOWER fraction generation =0.04475125900582107
NGCC fraction generation =0.0
NGCC_AMINE fraction generation =0.0
NGCC_CRYO fraction generation =0.0
NGCT fraction generation =0.010024349873589102
NGCT_AMINE fraction generation =0.0
NGCT_CRYO fraction generation =0.0
COAL fraction generation =0.9452243911205898
COAL_AMINE fraction generation =0.0
COAL_CRYO fraction generation =0.0
NUCLEAR fraction generation =0.0
SOLAR fraction generation =0.0
WIND fraction generation =0.0
